## PS6 Due June 1 5:30pm


### Trees and Forests
The main task in this problem set is to implement a decision tree algorithm, and use this to predict Titanic survival. I also ask you to implement bagging and random forest. Note: this is about implementing the algorithm, not just using an existing library.
Before you start I recommend you to become familiar with (James, Witten, Hastie, and Tibshirani, 2015, ch 8) (available on canvas).

In [430]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from collections import defaultdict
%matplotlib inline

### 1 Explore the Data
First, load and explore the data. Feel free to copy-paste this step from your previous work if you already have explored the Titanic data.

1. Load the data. Make sure you know the coding of all variables. In particular, you should be aware if a variable is categorical or numeric. Explain the coding scheme if it's not obvious.
As most of the variables in the data are categorical, we cannot just compute means and correlations. Rather create a table along these lines where you list the averages for those who survived and those who died.

The idea is to provide the reader with the information about variables, how these are correlated with survival, and the main categories. The table above suggests that survivors had lower class numbers, i.e. they were more likely to travel in upper classes, and there were less men amont the survivors.

This involves quite a bit of manual work: for instance, if you list males, there is no need to list females. You should not list all homes and destinations (there are hundreds of these), but you may list a few more common ones. You should also consider which variables you want to list here (ticket number is probably not interesting, unless it tells something like 􏰀how close were you to the boats􏰁).

2. Create such a summary table. You may add more statistics you consider interesting to this table.

In [431]:
#load the data
titanic = pd.read_csv('titanic.csv.bz2', na_values=['NA'])
titanic.head()

#coding of the variables:
#pclass = passenger class (1,2,3), categorical
#name = string
#sex (female/male), categorical
#sibsp = siblings or spouses aboard, numeric
#parch = parents or children aboard, numeric
#ticket = ticket #
#fare = passenger fare
#cabin = cabin
#embarked = port (cherbourg, queenstown, southampton), categorical
#boat = boat number
#body = body found y/n
#home.dest = where they were going

# some of the more interesting features that may be of relevance to us:
# passenger class; sex; fare (after reindexing categorically)
# and then a weird one--port of embarkment

#var = titanic[titanic['pclass'] == 3 & (titanic['survived'] == 1)]

#np.corrcoef(titanic.pclass, titanic.survived)

# ordering port of embarkation by people that borded the ship
# is using ordered categorical variables

cleanup_nums = {"embarked":     {"S": 1, "C": 2, "Q": 3},
                "sex": {"female": 0, "male": 1},
                # this part is kind of hard to figure out but my heuristic was sort of regional...
                
                # new york area
                "home.dest": {"Queens": 1, "Brooklyn": 1, "Bronx": 1, "Staten Island": 1,
                              "Manhattan": 1, "New York": 1, ", NY": 1, "NY": 1,
                              "NJ": 1, "New Jersey": 1, "Newark": 1,
                              
                              # boston and east coast
                              "Boston": 2, "Brighton": 2, "Roxbury": 2, "Allston": 2,
                              "Beacon Hill": 2, "Dorchester": 2,
                              "Suffolk": 2, "Massachusetts": 2, ", MA": 2, "MA": 2,
                              ", RI": 2, "RI": 2, ", DE": 2, ", VT": 2, ", ME": 2, ", CT": 2,
                              
                              # uk
                              "London": 3, ", UK": 3, "England": 3,
                              
                              # canada
                              "Montreal": 4, "Winnipeg": 4, ", ON": 4, "Toronto": 4,
                              "Halifax": 4,
                             
                              # paris/france
                              "Paris": 5, "France": 5,
                              
                              # chicago, indiana, ohio, pennsylvania
                              "Chicago": 6, ", IL": 6, "Illinois": 6, ", IN": 6,
                              "Akron": 6, ", OH": 6, "Ohio": 6, "OH": 6, ", Indiana": 6,
                              "Philadelphia": 6, ", PA": 6, "Pennsylvania": 6, "PA": 6,
                              
                              # ireland
                              "Belfast": 7, "Ireland": 7,
                              
                             # other places outside the US
                              "Mexico": 8, "Syria": 8, "Switzerland": 8, "Norway": 8,
                              "Sweden": 8, "Finland": 8, "Greece": 8, "Uruguay": 8,
                              
                              # california, washington, oregon
                              "California": 9, ", CA": 9, "CA": 9, ", WA": 9,
                              "Seattle": 9, ", OR": 9, "San Francisco": 9,
                              "Spokane": 9
                             }
               }


# this addresses about half of the different locations in the dataset
titanic.replace(cleanup_nums, regex=True, inplace=True)

# this line converts all of the places that are not in the list above to 0
titanic['home.dest'] = titanic['home.dest'].convert_objects(convert_numeric=True).fillna(0)

titanic['embarked'].fillna(1, inplace=True)

# we don't care about the name of the person, their ticket number, and boat/body
# so let's drop them
titanic = titanic.drop(['name', 'ticket', 'cabin', 'boat', 'body'], axis=1)

# replace nan values with mean for age
titanic['age'].fillna((titanic['age'].mean()), inplace=True)

# replace nan values with mean for fare
titanic['fare'].fillna((titanic['fare'].mean()), inplace=True)

sur = pd.DataFrame(titanic[titanic['survived'] == 1].mean()).reset_index()
dro = pd.DataFrame(titanic[titanic['survived'] == 0].mean()).reset_index()

description = pd.merge(sur, dro, on='index')
description.columns = ['variable', 'survived (mean val)', 'died (mean val)']
description

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use Series.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


,variable,survived (mean val),died (mean val)
0,pclass,1.962000,2.500618
1,survived,1.000000,0.000000
2,sex,0.322000,0.843016
3,age,29.058812,30.389368
4,sibsp,0.462000,0.521632
5,parch,0.476000,0.328801
6,fare,49.361184,23.366119
7,embarked,1.476000,1.343634
8,home.dest,1.798000,1.417800


Interpreting this data, we find that the person who survived was of a lower class number (higher class status), female, younger, had fewer siblings on board, more parents or children on board, paid a higher fare, and was going closer to Boston (or elsewhere) than people who died.

People who died were a lower class, male, older, had a sibling on board, had fewer parents or children on board, paid a lower fare, and was probably going to New York.

### 2 Implement decision tree
Now let's move step-by-step. Let's stay with ordinal variables only, i.e. such variables that can be ordered. You can order a) all binary variables (such as sex); b) all continuous variables, such as age; and c) ordered categorical variables, such as pclass (1st, 2nd, 3rd, but note that for datasets where pclass also includes crew, we cannot order it).
In order to simplify the evaluation, we split data into training and testing samples and avoid cross- validation.

#### 2.1 Prepare data
1. Now select the variables you are going to use. Use those that may have in􏰄uenced survival (such as class), or those that may be otherwise correlated with survival (like fare).
Note: do not include explanatory variables that are result of survival/death, namely boat number and body number.

2. Split your data into training/testing groups (80/20% or so).

In [432]:
# when i was cleaning and structuring the data in the cell above, i ended up selecting
# the different features that i care about for analysis.
#
# we're gonna try splitting based on all of the features listed here...

np.random.seed(11)
X = titanic.drop(['survived'], axis=1)
y = titanic['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20)
X_train.head()

,pclass,sex,age,sibsp,parch,fare,embarked,home.dest
660,3,0,24.000000,0,3,19.2583,2.0,1.0
1022,3,1,29.881135,0,0,7.8958,1.0,0.0
824,3,1,38.000000,0,0,7.0500,1.0,0.0
703,3,1,21.000000,0,0,7.7500,3.0,6.0
191,1,1,30.000000,0,0,45.5000,1.0,1.0


#### 2.2 Implement the decision tree

Now we'll get serious. Your task is to implement decision tree by recursive splitting including all the variables you selected above. The tree should predict survival. You'll work only on the training set here.

1. Compute the entropy at the root, based of the total number of survivors and victims. You will use this number to compute entropy gain further below.
Next, let's 􏰃nd the optimal split according to age:

2. create an ordered vector of unique age values (a1,a2,...,ak). These will form the potential split points for age. How many di􏰂erent age values do you 􏰃nd in your training data?
Note: remember that there are missing age values!

3. Split the (training) data into two groups:
(a) age 􏰅 a1 and (b) age > a1.

4. Compute the entropy of this split.

5. Now repeat these steps with all possible age splits. Find the best age boundary, the split that gives you the lowest entropy.
This will be your 􏰃rst split in case you decide to start splitting your tree along age. But perhaps another variable is better? Let's check it out!

6. Repeat the previous steps over all explanatory variables in your data. For each variable, store (and print) the best split value and the corresponding entropy.

7. Now pick the feature that gives the largest entropy gain. This gives you the 􏰃rst split (tree stump). Show the name of the feature, the resulting entropy, and the split position.

Print the survived/non-survived percentage in both branches. 

And now the most important and complex part. Repeat the previous steps over-and-over again using data in both branches you split as inputs in the next step until your tree is ready.

In [433]:
# calculating entropy of total survivors vs. victims

def calc(num):
    # here's how we calculate entropy
    # to prevent divide by zero errors
    # in the log function we make an if statement
    # that returns 0
    if num != 0:
        return -num * np.log2(num)
    else:
        return 0

    
    
def entropy(labels):
    
    if len(labels)==0:
        return 0
    
    a = np.sum(labels)/len(labels)
    b = 1-a
    if a == 0 or b == 0:
        return 0
    ent = (-a*np.log2(a)) - (b*np.log2(b))
    return ent

lived = sum(y_train) / float(len(y_train))
drowned = (len(y_train) - sum(y_train))/ float(len(y_train))

# and this calculates the entropy for the two different populations
rootentropy = calc(lived) + calc(drowned)
# apparently the training data tells us a lot
rootentropy

0.955396536466977

In [434]:
# define a function that returns the best split points
# ordered by information gain
#

def df_split(xdata, col, value):
    part1 = xdata[xdata[col] <= value]
    part2 = xdata[xdata[col] > value]
    return part1, part2

def uniq(xdata, col):
    return xdata[col].unique().tolist()

def optimal_split(xdata, ydata, col, p=False, f=False, sp=False, rc=False, total=False):
    vec = xdata[col].unique()
    split = {}
    for value in vec:
        # create the axial/split point
        part1, part2 = df_split(xdata, col, value)
        
        # use index of survival label to look up survival, and get truth value
        # gotta look up part1 indices that correspond with survived and pass that into entropy
        val1 = ydata.index.isin(part1.index)
        val2 = ydata.index.isin(part2.index)
        
        prop1 = sum(val1) / len(ydata)
        prop2 = sum(val2) / len(ydata)
        
        # the current issue is the entropy function  takes the length of the data passed in
        # and sums it since it's really just a boolean array
        split[value] = prop1 * entropy(ydata[val1]) + prop2 * entropy(ydata[val2])

    if p==True:
        print('category, entropy for', col)
        print(min(split.items()))
        print('\n')
    if f==True:
        return col, min(split.items())
    if sp==True:
        return min(split.keys())
    if rc==True:
        return col, min(split.items()), prop1, prop2
    if total==True:
        return split
    return min(split.values())

In [435]:
# this function finds all the best entropies
seq = [ optimal_split(X_train, y_train, i, p=False) for i in X_train.columns ]
seq
[ optimal_split(X_train, y_train, i, p=True) for i in X_train.columns ]
[ (seq[i], rootentropy - seq[i]) for i in range(len(seq))]

category, entropy for pclass
(1, 0.8973234274349375)


category, entropy for sex
(0, 0.765528794657654)


category, entropy for age
(0.1667, 0.9540487455366274)


category, entropy for sibsp
(0, 0.9490142995274883)


category, entropy for parch
(0, 0.9375913132284708)


category, entropy for fare
(0.0, 0.9505051748854196)


category, entropy for embarked
(1.0, 0.9371923556509358)


category, entropy for home.dest
(0.0, 0.9375761256784193)




[(0.8940031567890405, 0.061393379677936455),
 (0.765528794657654, 0.18986774180932298),
 (0.9443194222599807, 0.011077114206996308),
 (0.9432053023479049, 0.01219123411907208),
 (0.9375913132284708, 0.017805223238506196),
 (0.89642445665317, 0.05897207981380692),
 (0.9371923556509358, 0.018204180816041138),
 (0.9375761256784193, 0.017820410788557672)]

Based on the output here it seems apparent that the best split is on sex(0), with an entropy gain of almost 20%. It's the second index in the list there...

In [436]:
optimal_split(X_train, y_train, 'sex', rc=True)
# the second part is the ratio; apparently sex classifies all of our traning data here
# probably a good place to start ;)

('sex', (0, 0.765528794657654), 1.0, 0.0)

In [437]:
def buildtree(xdata, ydata, stop):
    if len(xdata)==0: return 0
 
    bestgain = 0.0
    bestcrit = None
    bestset = None
    
    split = {}
    info = entropy(ydata)
    
    for col in xdata.columns:
    
    # get the unique vals for each column in the data
        vec = xdata[col].unique()
        
        # for those vals
        for value in vec:
            # split the dataset
            (part1, part2) = df_split(xdata, col, value)
            
            # check indices for survival
            val1 = ydata.index.isin(part1.index)
            val2 = ydata.index.isin(part2.index)
            
            # calc proportion and entropy
            prop1 = sum(val1) / len(ydata)
            prop2 = sum(val2) / len(ydata)
            ent = prop1 * entropy(ydata[val1]) + prop2 * entropy(ydata[val2])
            split[value] = ent
        
            # calculate infogain
            infogain = info - min(split.items())[1]

            if infogain > bestgain and len(part1) > stop and len(part2) > stop:
                bestgain = infogain
                
                bestcrit = (col, value)
                
                bestset = (part1, part2)
                
    while bestgain > 0:
        # Recurse and move to the next level of the tree
        split0 = buildtree(bestset[0], ydata, stop)
        split1 = buildtree(bestset[1], ydata, stop)
        # And build out new nodes
        return [bestcrit[0], bestcrit[1], split0, split1]
    return [sum(val1+val2)/len(ydata)]

In [438]:
bigtree = buildtree(X_train, y_train, 0)
# i couldnt work out how to pretty print this, but the ratio we see before a list
# suggests the percentage of survivors in that list
bigtree

['sex',
 0,
 ['pclass',
  1,
  ['home.dest',
   0.0,
   ['age',
    35.0,
    ['age',
     30.0,
     ['age',
      19.0,
      ['age',
       18.0,
       ['age',
        16.0,
        ['age', 15.0, [0.0009551098376313276], [0.0009551098376313276]],
        ['age', 17.0, [0.0009551098376313276], [0.0009551098376313276]]],
       [0.0009551098376313276]],
      ['age',
       29.0,
       ['age',
        26.0,
        [0.0009551098376313276],
        ['fare', 211.3375, [0.0009551098376313276], [0.0009551098376313276]]],
       ['age',
        29.8811345124283,
        [0.0009551098376313276],
        ['fare',
         56.9292,
         ['fare', 31.0, [0.0009551098376313276], [0.0009551098376313276]],
         ['fare', 93.5, [0.0009551098376313276], [0.0009551098376313276]]]]]],
     ['age',
      33.0,
      ['age',
       31.0,
       [0.0009551098376313276],
       ['age',
        32.0,
        [0.0009551098376313276],
        ['sibsp', 0, [0.0009551098376313276], [0.0009551098376313

When is the tree ready? There are a number of options:

i. you get a get a pure leaf (only survivors or victims in the leaf)

ii. you run out of data (number of observations in your leaf is too small, say n < 5)

iii. you run out of variables: there are no more features to split along.

iv. you still have features left, but there is no variation. For instance, you extract a small age group
and 􏰃nd that there are no men in there.

But it is not enough to do the splitting and all that, you have also to create a tree, and later be able to use this for predictions. I recommend to use nested lists for this, but you may also use some sort of tree data structures, or graphs if you wish. A potential way to do it in form of lists is (R syntax for more verbosity):

*list(feature, splitpoint, split0, split1)*

for instance

*list(age, 20, dl, dr)*

where dl and dr are the respective subtrees (left and right branch) in case the tree branches here, and list of lenght one is a leaf, for instance

*list(0.8)*

means the probability of survival for this branch is 0.8. Now you can distinguish between leafs and
nodes by looking at the length of the corresponding subtree.

8. Create such a recursive function we discussed in the class that takes in data and returns a tree.

9. Visualize the two top levels of branches of your tree. Try to make the result readable (I don't ask anything like fancy graphical results) but ensuring that one decision is on a single line, and adding a little manual indentation will make a large step toward making the tree readable.
Comment the outcome. Does it make sense? Which variables seem to be more important?

10. Predict (based on training data) using your tree. Compute accuracy, precision and recall.
Note: you have to create a function that walks through the nested list based on a single observation. As the decisions di􏰂er between individual observations, we cannot easily vectorize the operation. A set of loops is necessary (can be done in parallel).

In [439]:
smalltree = buildtree(X_train, y_train, 50)
smalltree

['sex',
 0,
 ['pclass',
  1,
  ['sibsp', 0, [0.054441260744985676], [0.05348615090735435]],
  ['pclass',
   2,
   [0.08118433619866285],
   ['sibsp',
    0,
    ['embarked', 1.0, [0.04871060171919771], [0.049665711556829036]],
    [0.07067812798471824]]]],
 ['pclass',
  1,
  ['fare', 45.5, [0.0659025787965616], [0.06876790830945559]],
  ['age',
   19.0,
   [0.07545367717287488],
   ['fare',
    18.0,
    ['fare',
     7.875,
     ['fare', 7.7417, [0.08691499522445081], [0.05826170009551098]],
     ['fare',
      8.6625,
      [0.09933142311365807],
      ['home.dest', 0.0, [0.06494746895893028], [0.05348615090735435]]]],
    [0.06876790830945559]]]]]

*The variables that seem to be most important are first sex, then passenger class, then age, then fare, and then my overengineered variable location.*

In [445]:
def classify(tree, observation):
    if len(tree) == 1:
        return True
    else:
        if observation[tree[0]] <= tree[1]:
            return classify(tree[2], observation)
        else:
            return False
        if observation[tree[0]] > tree[1]:
            return classify(tree[3], observation)
        else:
            return False

def dfclassify(xdata, tree):
    preds = []
    for i in xdata.index:
        predict = classify(tree, xdata.loc[i])
        preds.append(predict)
    return preds

ylabels = dfclassify(X_train, bigtree)

In [446]:
# roc metrics, borrowed from altan orhon
def roc_metrics(Y, preds):  
    confmat = confusion_matrix(Y, preds)
    c = confmat.sum().sum()
    accuracy = confmat.diagonal().sum()/c
    sensitivity = confmat[0,0]/(confmat[0,:].sum())
    specificity = confmat[1,1]/(confmat[1,:].sum())
    precision = confmat[1,1]/(confmat[:,1].sum())
    return {'accuracy' : accuracy, 'sensitivity' : sensitivity,
            'specificity' : specificity, "precision" : precision}

roc_metrics(y_train, ylabels)
# the values for the roc metrics are shown below
# as ott said in the assignment, pretty mediocre
# smells like overfit to me.

{'accuracy': 0.6246418338108882,
 'precision': 1.0,
 'sensitivity': 1.0,
 'specificity': 0.0025380710659898475}

### 3 Bagging and Random Forests
So far we just estimated a single tree without any pruning. The test results were probably mediocre. Now the task is to improve the performance using bagging and random forests.
#### 3.1 Bagging
First, let's do bagging (bootstrapped aggregating). It means creating a number of trees, using a di􏰂erent bootstrapped dataset each time. The 􏰃nal prediction will be done by majority voting between individual predictions.
To get a bootstrapped dataset you start with your (training) data, and each time you select N obser- vations out of N with replacement (see James, Witten, Hastie, and Tibshirani, 2015, section 5.2). This means we'll have some observations in the data repeatedly while some are missing.
1. Create a small number B of bagging trees (B = 5 is a good choice).
Note: as your tree algorithm may be slow, use this to measure and boost the speed. Suggestions to improve the speed are a) use fewer features (but keep age, class, sex); b) use a larger minimum number of observations for splits (say, 100); and c) run your tree-growing code in parallel.
2. predict (on test data) the survival according to each individual tree.
3. Show some aggregate statistics: in how many cases all trees agree?
4. Find your 􏰃nal prediction: the majority vote over all trees. Compute accuracy, precision, recall. Did you get better results than in case of the single tree?
True bagging involves many more iterations, potentially thousands. How many you should use to get substantial improvement. Or maybe bagging does not help at all here?
5. Repeat the process above with a large range of B, say, between 1 and 300. (Hint: you don't have to test every single number between 1 and 300. Choose numbers that make this not too slow on your computer.) For each B, compute accuracy, precision, recall.
6. Showona􏰃gurehowA,P,RdependontheB.
7. Discuss your 􏰃ndings. What is the optimal bag size B?

In [447]:
sX_train = X_train[['pclass', 'age', 'sex']]
sX_test = X_test[['pclass', 'age', 'sex']]

# Create a random subsample from the dataset with replacement
def bag(dataset, n=None):
    return dataset.sample(n=len(dataset), replace=True)

In [471]:
# results for bagging with 5 trees
# as you can see the results are pretty good
#
fivebags = pd.DataFrame()
for i in range(5):
    metric = []
    newdata = bag(sX_train)
    trees = buildtree(newdata, y_train, stop=200)
    for i in sX_test.index:
        var = classify(trees, sX_test.loc[i])
        metric.append(var)
    calc = roc_metrics(y_test, metric)
    fivebags = fivebags.append([calc])

fivebags.describe()

,accuracy,precision,sensitivity,specificity
count,5.000000,5.000000,5.000000,5.000000
mean,0.812977,0.813187,0.891026,0.698113
std,0.000000,0.000000,0.000000,0.000000
min,0.812977,0.813187,0.891026,0.698113
25%,0.812977,0.813187,0.891026,0.698113
50%,0.812977,0.813187,0.891026,0.698113
75%,0.812977,0.813187,0.891026,0.698113
max,0.812977,0.813187,0.891026,0.698113


In [472]:
# results for bagging with 50 trees
# as you can see the results are solid looking
#
fiftybags = pd.DataFrame()
for i in range(50):
    metric = []
    newdata = bag(sX_train)
    trees = buildtree(newdata, y_train, stop=200)
    for i in sX_test.index:
        var = classify(trees, sX_test.loc[i])
        metric.append(var)
    calc = roc_metrics(y_test, metric)
    fiftybags = fiftybags.append([calc])

fiftybags.describe()

,accuracy,precision,sensitivity,specificity
count,50.000000,50.000000,50.000000,50.000000
mean,0.810534,0.812737,0.892051,0.690566
std,0.017273,0.003180,0.007252,0.053367
min,0.690840,0.790698,0.891026,0.320755
25%,0.812977,0.813187,0.891026,0.698113
50%,0.812977,0.813187,0.891026,0.698113
75%,0.812977,0.813187,0.891026,0.698113
max,0.812977,0.813187,0.942308,0.698113


In [473]:
# results for bagging with 1 tree
# as you can see the results are pretty good
#
onebag = pd.DataFrame()
for i in range(1):
    metric = []
    newdata = bag(sX_train)
    trees = buildtree(newdata, y_train, stop=200)
    for i in sX_test.index:
        var = classify(trees, sX_test.loc[i])
        metric.append(var)
    calc = roc_metrics(y_test, metric)
    onebag = onebag.append([calc])

onebag.describe()

,accuracy,precision,sensitivity,specificity
count,1.000000,1.000000,1.000000,1.000000
mean,0.812977,0.813187,0.891026,0.698113
std,NaN,NaN,NaN,NaN
min,0.812977,0.813187,0.891026,0.698113
25%,0.812977,0.813187,0.891026,0.698113
50%,0.812977,0.813187,0.891026,0.698113
75%,0.812977,0.813187,0.891026,0.698113
max,0.812977,0.813187,0.891026,0.698113


I don't have a plot for you, but check out these awesome charts! This is *much* better than single tree model, but it slows down my computer a lot, to the point that I have to walk away for about fifteen-twenty minutes. And this is is a two year old computer. In any case, I suspect that as we increase B, we produce more prediction vectors that are identical or close to identical, which means less deviation, which means a more precise prediction. It's like law of large numbers all over again!!

### 3.2 Random Forests
Boosting helps but it does not use much more information than a single tree. Random forests approaches the issue by only considering a random sample of possible features as split candidates, where the sample should be less than all potential candidates (otherwise we are back to bagging). A good choice is to choose sample size m ≈ √p where p is the total number of features to consider.
1. Implement random tree algorithm by changing your previous growTree algorithm. We refer the new version as growForestTree.
Note: depending on how you implemented the data structures and the prediction algorithm, you may have to adjust the prediction function too.
2. Create random forests of di􏰂erent size B. Ideally you want to go up to B = 1000, but you may need to keep the number much smaller in order to 􏰃nish in a reasonable time. For each B, compute accuracy, precision, recall.
3. Show on a 􏰃gure how A, P, R depend on the B. Include the bagging outcomes on the same 􏰃gure too.
4. Discuss your 􏰃ndings. What is the optimal forest size? Are random forests superior to bagging?

In [474]:
# so let's reimplement...
#
#
def growForestTree(xdata, ydata, stop):
    if len(xdata)==0: return 0
 
    bestgain = 0.0
    bestcrit = None
    bestset = None
    
    split = {}
    info = entropy(ydata)
    
    for col in xdata.columns:
    
    # get the unique vals for each column in the data
    #
    # this is how i modified my code to take
    # a random subset of features
        feat = xdata[col].unique()
        vec = np.random.choice(feat, np.round(int(np.sqrt(len(feat)))))
        #
        #
        # for those vals
        for value in vec:
            # split the dataset
            (part1, part2) = df_split(xdata, col, value)
            
            # check indices for survival
            val1 = ydata.index.isin(part1.index)
            val2 = ydata.index.isin(part2.index)
            
            # calc proportion and entropy
            prop1 = sum(val1) / len(ydata)
            prop2 = sum(val2) / len(ydata)
            ent = prop1 * entropy(ydata[val1]) + prop2 * entropy(ydata[val2])
            split[value] = ent
        
            # calculate infogain
            infogain = info - min(split.items())[1]

            if infogain > bestgain and len(part1) > stop and len(part2) > stop:
                bestgain = infogain
                
                bestcrit = (col, value)
                
                bestset = (part1, part2)
                
    while bestgain > 0:
        # Recurse and move to the next level of the tree
        split0 = buildtree(bestset[0], ydata, stop)
        split1 = buildtree(bestset[1], ydata, stop)
        # And build out new nodes
        return [bestcrit[0], bestcrit[1], split0, split1]
    return [sum(val1+val2)/len(ydata)]

In [476]:
# results for random forest with 1 tree
# as you can see the results are solid looking
#
randomforestone = pd.DataFrame()
for i in range(1):
    metric = []
    newdata = bag(sX_train)
    trees = growForestTree(sX_train, y_train, stop=100)
    for i in sX_test.index:
        var = classify(trees, sX_test.loc[i])
        metric.append(var)
    calc = roc_metrics(y_test, metric)
    randomforestone = randomforestone.append([calc])

randomforestone.describe()

,accuracy,precision,sensitivity,specificity
count,1.000000,1.000000,1.000000,1.000000
mean,0.698473,0.935484,0.987179,0.273585
std,NaN,NaN,NaN,NaN
min,0.698473,0.935484,0.987179,0.273585
25%,0.698473,0.935484,0.987179,0.273585
50%,0.698473,0.935484,0.987179,0.273585
75%,0.698473,0.935484,0.987179,0.273585
max,0.698473,0.935484,0.987179,0.273585


In [478]:
# results for random forest with 5 trees
# as you can see the results are solid looking
#
randomforestfive = pd.DataFrame()
for i in range(5):
    metric = []
    newdata = bag(sX_train)
    trees = growForestTree(sX_train, y_train, stop=100)
    for i in sX_test.index:
        var = classify(trees, sX_test.loc[i])
        metric.append(var)
    calc = roc_metrics(y_test, metric)
    randomforestfive = randomforestfive.append([calc])

randomforestfive.describe()

,accuracy,precision,sensitivity,specificity
count,5.000000,5.000000,5.000000,5.000000
mean,0.724427,0.930521,0.982051,0.345283
std,0.035539,0.006796,0.007022,0.098177
min,0.698473,0.923077,0.974359,0.273585
25%,0.698473,0.923077,0.974359,0.273585
50%,0.698473,0.935484,0.987179,0.273585
75%,0.763359,0.935484,0.987179,0.452830
max,0.763359,0.935484,0.987179,0.452830


In [479]:
# results for random forest with 50 trees
# as you can see the results are solid looking
#
randomforestfifty = pd.DataFrame()
for i in range(50):
    metric = []
    newdata = bag(sX_train)
    trees = growForestTree(sX_train, y_train, stop=100)
    for i in sX_test.index:
        var = classify(trees, sX_test.loc[i])
        metric.append(var)
    calc = roc_metrics(y_test, metric)
    randomforestfifty = randomforestfifty.append([calc])

randomforestfifty.describe()

,accuracy,precision,sensitivity,specificity
count,50.000000,50.000000,50.000000,50.000000
mean,0.679237,0.876036,0.901538,0.352075
std,0.086474,0.160564,0.269927,0.200110
min,0.404580,0.404580,0.000000,0.207547
25%,0.698473,0.930300,0.982372,0.273585
50%,0.698473,0.935484,0.987179,0.273585
75%,0.698473,0.935484,0.987179,0.313679
max,0.763359,0.948718,0.987179,1.000000


In [498]:
dflist = [onebag, fivebags, fiftybags, 
          randomforestone, randomforestfive, 
          randomforestfifty]

def makedf(dflist):
    newdf = pd.DataFrame()
    for i in dflist:
        newdf = newdf.append(i.describe().loc['mean'])
    return newdf

apss = makedf(dflist)
apss = apss.T
apss.columns = ['one (bagging)', 'five (bagging)', 'fifty (bagging)', 
          'one (rf)', 'five (rf)', 'fifty (rf)']

apss

,one (bagging),five (bagging),fifty (bagging),one (rf),five (rf),fifty (rf)
accuracy,0.812977,0.812977,0.810534,0.698473,0.724427,0.679237
precision,0.813187,0.813187,0.812737,0.935484,0.930521,0.876036
sensitivity,0.891026,0.891026,0.892051,0.987179,0.982051,0.901538
specificity,0.698113,0.698113,0.690566,0.273585,0.345283,0.352075


I saw the most gains across the board at five forests, but I suspect that it would increase/level off with more forests. It kind of depends on the randomness, right?

I am really surprised that rf is so inefficient ocmpared to bagging, in terms of accuracy. I wonder if there is some kind of best subset selection for features that end up getting 'forested' (to coin a term).

I can assert from podcasts I have listened to (specifically 'Partially Derivative' and 'Talking Machines') that since random forests reduce correlation across trees, but bagging improves variance by averaging across trees, I would suspect that between the two there's an optimal threshold where one becomes more efficient and the other ceases to be.

Thanks for a great quarter Ott and Gos! This course overall really squished, streched, and stomped on my brain. It was really challenging in the best possible ways. :)
Adieu!!